<a href="https://colab.research.google.com/github/Raoufmamedov/PET_PROJECTS/blob/main/Jupyter_Notebook_AI_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D0%BD%D0%B8%D0%BA_%D0%BD%D0%B0_%D0%B1%D0%B0%D0%B7%D0%B5_LLM_%D1%81_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
# <nbformat>5</nbformat>

In [33]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [53]:
!ls /content/drive/MyDrive/; grep 'Jupyter Notebook: AI-помощник на базе LLM с RAG для запросов к базе знаний.ipynb'

 00_preprocessing.ipynb
 00_stopwords.ipynb
 01_dot_product_attention.ipynb
 01_input_pipeline.ipynb
 01_sentiment_with_flair.ipynb
 02_build_and_train.ipynb
 02_sentiment_with_transformers.ipynb
 20150912_113241.jpg
 2_5278745515247274913.pptx
 2_5285040378230080100.epub
 4_5785090896378726757.pdf
'AI assisted SelfEd, NLP, LLM'
 akafist4.gdoc
 Amsterdam.gmap
'API #1'
 appsheet
 AZ16.gmap
 Backup.zip
'Baku 4.1.gmap'
'BAKU VISIT.gmap'
'BAKU VISIT ver4.gmap'
 BOOKING
 Brussel.gmap
'BU9W7187_новый размер.JPG'
 BU9W7240.JPG
'BU9W7328_новый размер.JPG'
'BU9W7409_новый размер.JPG'
'BU9W7410_новый размер.JPG'
'BU9W7480_новый размер.JPG'
'BU9W7481_новый размер.JPG'
'BU9W7511_новый размер.JPG'
'BU9W7512_новый размер.JPG'
'BU9W7517_новый размер.JPG'
'BU9W7598_новый размер.JPG'
'BU9W7627_новый размер.JPG'
'BU9W7646_новый размер.JPG'
'BU9W7654_новый размер.JPG'
'BU9W7692_новый размер.JPG'
'BU9W7702_новый размер.JPG'
'BU9W7771_новый размер.JPG'
'BU9W7780_новый размер.JPG'
'BU9W7790

In [61]:

import nbformat

path = '/content/drive/MyDrive/Jupyter Notebook: AI-помощник на базе LLM с RAG'  # без начального слеша

with open(path, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=nbformat.NO_CONVERT)

if 'widgets' in nb['metadata']:
    del nb['metadata']['widgets']

with open(path, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print("Готово. Виджеты удалены.")

Готово. Виджеты удалены.


In [8]:
import os
os.listdir()

['.config', 'sample_data']

# **Jupyter Notebook: AI-помощник на базе LLM с RAG для запросов к базе знаний**

## **1. Введение**

### Данный ноутбук посвящен разработке интеллектуального помощника, использующего большие языковые модели (LLM) и Retrieval-Augmented Generation (RAG) для ответов на вопросы на основе предоставленной базы знаний.

### **Бизнес-проблема:** Компании часто имеют обширную внутреннюю документацию, к которой трудно получить быстрый и точный доступ. AI-помощник с RAG может предоставлять контекстно-зависимые ответы, улучшая эффективность и удовлетворенность.

### **Цель проекта:** Построить RAG-систему, способную отвечать на вопросы на основе реального текстового корпуса, используя бесплатные ресурсы Google Colab. В качестве примера мы сформируем базу знаний из англоязычного текста Библии короля Якова (KJB), опубликованного на сайте проекта Project Gutenberg. Это большой корпус, состаящий из 39 книг, и он хорошо подходит в качестве коллекции документов различающихся стилем и манерой повествования, а также содержащих упоминание разных личностей имеющих одно и то же имя.

### Источник данных: "The King James Version of the Bible" (Библия короля Якова)
### URL: https://www.gutenberg.org/files/10/10-0.txt

In [ ]:
# ## 2. установка и импорт необходимых библиотек, настройка среды
# Здесь мы импортируем все библиотеки, которые потребуются для проекта.
# Если ноутбук запускается в Google Colab, убедитесь, что используется среду с GPU (T4).

!pip install transformers sentence-transformers langchain langchain-community faiss-cpu accelerate bitsandbytes -q
!pip install pypdf -q # Для чтения PDF, если понадобится


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import requests # Для загрузки данных
from tqdm.notebook import tqdm # Для прогресс-баров
from google.colab import userdata
import os

# Для RAG
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import HuggingFacePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch
from langchain_core.documents import Document

In [ ]:
# Проверяем доступность GPU
if torch.cuda.is_available():
    print(f"GPU доступен: {torch.cuda.get_device_name(0)}")
    device = "cuda"
else:
    print("GPU не доступен, будет использоваться CPU.")
    device = "cpu"


GPU доступен: Tesla T4


### Осуществляем загрузку данных и их очистку с удалением метаданных и сохраняем текст для дальнейшей обработки


In [ ]:
print("Загрузка текста книги из Project Gutenberg...")
book_url = "https://www.gutenberg.org/files/10/10-0.txt"
response = requests.get(book_url)
response.raise_for_status() # Проверка, успешности запроса

raw_text = response.text
print(f"Текст успешно загружен. Размер текста: {len(raw_text)} символов.")

# Очистка текста: удаляем метаданные Project Gutenberg в начале и конце
start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK 10 ***"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK 10 ***"

start_index = raw_text.find(start_marker)
end_index = raw_text.find(end_marker)

if start_index!= -1 and end_index!= -1:
    cleaned_text = raw_text[start_index + len(start_marker):end_index].strip()
    print(f"Текст очищен от метаданных. Размер очищенного текста: {len(cleaned_text)} символов.")
else:
    cleaned_text = raw_text.strip()
    print("Метаданные не найдены, используется весь текст.")

# Сохраняем очищенный текст в файл для дальнейшей обработки
with open("KJB.txt", "w", encoding="utf-8") as f:
    f.write(cleaned_text)
print("Очищенный текст сохранен в 'KJB.txt'.")

Загрузка текста книги из Project Gutenberg...
Текст успешно загружен. Размер текста: 4432276 символов.
Текст очищен от метаданных. Размер очищенного текста: 4432176 символов.
Очищенный текст сохранен в 'KJB.txt'.


## **Предварительная обработка текста и генерация эмбеддингов**

### На этом этапе мы подготовим текстовые данные для RAG-системы: чанкируем текст, сгенерируем векторные представления (эмбединги), индексируя их в векторной базе данных FAISS, которая позволит нам эффективно искать сходство между векторами.

In [ ]:
from langchain_core.documents import Document

# Обёртывание текста в объект Document
documents = [Document(page_content=cleaned_text, metadata={"source": "your_source"})]

# Разбиение
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50,
    length_function=len
)
chunks = text_splitter.split_documents(documents)  # Список чанков

print(f"Текст разбит на {len(chunks)} чанков.")
print("Пример чанка:")
print(chunks[25].page_content[:1000] + "...")

Текст разбит на 12309 чанков.
Пример чанка:
field, and every fowl of the air; and brought them unto Adam to see
what he would call them: and whatsoever Adam called every living
creature, that was the name thereof.

2:20 And Adam gave names to all cattle, and to the fowl of the air,
and to every beast of the field; but for Adam there was not found an
help meet for him....


### Для создания из чанков индексируемых в FAISS эмбедингов мы будем использовать небольшую и эффективную модель семантического поиска **all-MiniLM-L6-v2**. Для последующего локального использования в API, мы сохраним базу данных локально

In [ ]:
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs={'device': device})

In [ ]:
# 4.3. Индексация в векторной базе данных (FAISS)
vector_store = FAISS.from_documents(chunks, embeddings)

# Сохраняем векторную базу
vector_store_path = "/tmp/faiss_index_KJB"
vector_store.save_local(vector_store_path)
print(f"Путь к хранилищу базы данных FAISS: {vector_store_path}")

Путь до векторной базы данных FAISS: /tmp/faiss_index_KJB


## **Настройка Большой Языковой Модели (LLM)**

### Генерировать ответы мы будем с помощью сравнительно небольшой и при этом  мощной модели **gemma-7b-it**, работающей даже на CPU применяя 4-битную квантизацию, чтобы уместить модель в память GPU и ускорить её.

In [ ]:
# import os
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
# from google.colab import drive

# # --- 1. Монтирование Google Drive ---
# # Это необходимо для доступа к вашим файлам на Google Drive
# print("Монтирование Google Drive...")
# drive.mount('/content/drive')
# print("Google Drive смонтирован.")

# # --- 2. Определение путей сохранения на Google Drive ---
# # Выберите путь на вашем Google Drive, где будут храниться модели.
# # Рекомендуется создать отдельную папку, например, 'colab_models'.
# drive_base_path = '/content/drive/MyDrive/colab_models'

# # Имя вашей LLM модели (например, "google/gemma-7b-it" или "meta-llama/Llama-3-8B-Instruct")
# llm_model_name = "google/gemma-7b-it" # Укажите здесь вашу выбранную модель

# # Создаем специфические пути для модели и токенизатора на Google Drive
# model_save_path = os.path.join(drive_base_path, llm_model_name.replace("/", "_") + "_model")
# tokenizer_save_path = os.path.join(drive_base_path, llm_model_name.replace("/", "_") + "_tokenizer")

# # --- 3. Настройка квантизации (как у вас было) ---
# nf4_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# # --- 4. Загрузка или скачивание модели и токенизатора ---
# tokenizer = None
# model_llm = None

# # Проверяем, существует ли модель уже на Google Drive
# if os.path.exists(model_save_path) and os.path.exists(tokenizer_save_path):
#     print(f"Модель и токенизатор '{llm_model_name}' найдены на Google Drive. Загрузка...")
#     try:
#         tokenizer = AutoTokenizer.from_pretrained(tokenizer_save_path)
#         model_llm = AutoModelForCausalLM.from_pretrained(
#             model_save_path,
#             quantization_config=nf4_config,
#             torch_dtype=torch.bfloat16,
#             device_map="auto"
#         )
#         print("Модель и токенизатор успешно загружены с Google Drive.")
#     except Exception as e:
#         print(f"Ошибка при загрузке с Google Drive: {e}. Попытка скачать из Hugging Face...")
#         # Если загрузка с Drive не удалась, удаляем неполные файлы и скачиваем заново
#         if os.path.exists(model_save_path):
#             os.rmdir(model_save_path) # Удалить, если была частичная загрузка
#         if os.path.exists(tokenizer_save_path):
#             os.rmdir(tokenizer_save_path) # Удалить, если была частичная загрузка

#         tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
#         model_llm = AutoModelForCausalLM.from_pretrained(
#             llm_model_name,
#             quantization_config=nf4_config,
#             torch_dtype=torch.bfloat16,
#             device_map="auto"
#         )
#         # После успешной загрузки из Hugging Face, сохраняем на Drive
#         print(f"Модель и токенизатор '{llm_model_name}' успешно скачаны. Сохранение на Google Drive...")
#         os.makedirs(model_save_path, exist_ok=True)
#         os.makedirs(tokenizer_save_path, exist_ok=True)
#         model_llm.save_pretrained(model_save_path)
#         tokenizer.save_pretrained(tokenizer_save_path)
#         print("Модель и токенизатор сохранены на Google Drive для будущего использования.")
# else:
#     print(f"Модель и токенизатор '{llm_model_name}' не найдены на Google Drive. Скачивание из Hugging Face...")
#     # Если на Drive нет, скачиваем из Hugging Face
#     tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
#     model_llm = AutoModelForCausalLM.from_pretrained(
#         llm_model_name,
#         quantization_config=nf4_config,
#         torch_dtype=torch.bfloat16,
#         device_map="auto"
#     )
#     # После успешной загрузки из Hugging Face, сохраняем на Drive
#     print(f"Модель и токенизатор '{llm_model_name}' успешно скачаны. Сохранение на Google Drive...")
#     os.makedirs(model_save_path, exist_ok=True)
#     os.makedirs(tokenizer_save_path, exist_ok=True)
#     model_llm.save_pretrained(model_save_path)
#     tokenizer.save_pretrained(tokenizer_save_path)
#     print("Модель и токенизатор сохранены на Google Drive для будущего использования.")

# # --- 5. Создание пайплайна (как у вас было) ---
# text_generation_pipeline = pipeline(
#     task="text-generation",
#     model=model_llm,
#     tokenizer=tokenizer,
#     max_new_tokens=256,
#     do_sample=True,
#     temperature=0.5, # Используйте свои последние настройки
#     top_k=75,
#     top_p=0.9,
#     model_kwargs={"quantization_config": nf4_config, "torch_dtype": torch.bfloat16}
# )

# print("Настройка RAG-системы завершена.")

In [ ]:
# Конфигурация для 4-битной квантизации
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Загрузка токенизатора и модели
# llm_model_name = "google/gemma-7b-it"
# llm_model_name = "meta-llama/Llama-3-8B-Instruct"
llm_model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct
print(f"Загрузка токенизатора для {llm_model_name}...")

Загрузка токенизатора для mistralai/Mistral-7B-Instruct-v0.2...


### Модель 'google/gemma-7b-it' является "gated repo" на Hugging Face. Для доступа к ней необходимо:
1. Перейти на страницу модели на Hugging Face (https://huggingface.co/google/gemma-7b-it) и принять условия использования.
2. Сгенерировать API токен в настройках Hugging Face (https://huggingface.co/settings/tokens).
3. Добавить этот токен в секреты Colab с именем HF_TOKEN.
4. Перезапустить сессию Colab (Runtime -> Restart session).

### Загружаем квантизировагнную модель LLM на доступное устройство, далее создаём пайплайн HuggingFace для генерации текста ответа.

In [ ]:
# Загружаем модель с Hugging Face.
from google.colab import userdata
import os

hf_token = userdata.get('HF_TOKEN')
if hf_token:
    os.environ["HF_TOKEN"] = hf_token
else:
    print("Токен Hugging Face не найден")

tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
print(f"Загрузка модели {llm_model_name} с 4-битной квантизацией...")
model_llm = AutoModelForCausalLM.from_pretrained(
    llm_model_name,
    quantization_config=nf4_config,
    torch_dtype=torch.bfloat16,
    device_map="auto") # Автозагрузка модели на доступное устройство (GPU/CPU)

# Создание пайплайна
text_generation_pipeline = pipeline(
    task="text-generation",
    model=llm_model_name,
    tokenizer=tokenizer, # Remove tokenizer argument when passing model name
    max_new_tokens=256, # Максимальное количество новых токенов в ответе
    do_sample=True,
   temperature=0.5, # Увеличено немного, для баланса между точностью и синтезом
    top_k=75,        # Оставляем как было
    top_p=0.9,       # Увеличено немного
    # num_beams=5,
    # repetition_penalty=1.1, # Для предотвращения повторений
    model_kwargs={"quantization_config": nf4_config, "torch_dtype": torch.bfloat16} # передача конфигурации квантизации
)
print("HuggingFace Pipeline для генерации текста настроен.")


# Оборачиваем пайплайн в LangChain LLM
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
print("LLM-пайплайн для LangChain настроен.")


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Загрузка модели mistralai/Mistral-7B-Instruct-v0.2 с 4-битной квантизацией...


...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


HuggingFace Pipeline для генерации текста настроен.
LLM-пайплайн для LangChain настроен.


/tmp/ipython-input-28-1598062982.py:37: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


### Конвейер RAG остоит из двух составляющих:

### 1.  **Retrieval (Получение):** Извлечение на основе запроса пользователя релевантных текстовых фрагментов из базы знаний.
### 2.  **Generation (Генерация):** Использование LLM для создания ответа, основываясь на извлеченном контексте.

### Создаём промпт, инструктирующий LLM как ей необходимо отвечать на вопросы исходя из контекста, а также RAG-цепочку, связывающую векторный поиск с генерацией,  а также ретривер для извлечения релевантного контента из векторной базы данных (задаётся количество выдаваемых единиц контента).


# Порядок операций в RAG-цепочке
### 1. Получение вопроса
### 2. Передаем вопроса ретриверу для извлечения контекста
### 3. Формирование промпта с контекстом и вопросом
### 4. Передача промпта в LLM для генерации ответа
### 5. Парсинг выводимого LLM текста в строку

In [ ]:
# Промпт
template = """Используйте только следующий фрагмент контекста, чтобы ответить на вопрос пользователя.

Инструкции:

Извлеките все релевантные факты и информацию, относящиеся к вопросу.

На основе этих фактов, сформулируйте краткий, точный и информативный ответ, что известно о той или иной личности, предмете, обстоятельствах, месте, действиях, о которых спрашивается.

Если контекст не содержит достаточной информации для прямого ответа на вопрос, укажите это.

Контекст:
{context}

Вопрос: {question}

Ответ:"""

RAG_PROMPT_TEMPLATE = PromptTemplate.from_template(template)

# Создание RAG-цепочки (LangChain)
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# Создаем ретривер из векторного хранилища
retriever = vector_store.as_retriever(search_kwargs={"k": 7})

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}

| RAG_PROMPT_TEMPLATE
| llm
| StrOutputParser()
)

print("Конвейер RAG успешно настроен.")

Конвейер RAG успешно настроен.


### Мы построили RAG систему. Самое время исппытать её и посмотреть что у нас получилось.

In [ ]:
# --- 7. Тестирование RAG-системы ---

print("\n--- Тестирование RAG-системы ---")

# Вопрос 1: Вопрос, на который есть ответ в тексте
question1 = "Who was Uriah the Hittite?"
print(f"Вопрос: {question1}")
response1 = rag_chain.invoke(question1)
print(f"Ответ: {response1}")
print("-" * 50)

# Вопрос 2: Вопрос о персонаже
question2 = "Name of first son of Adam?"
print(f"Вопрос: {question2}")
response2 = rag_chain.invoke(question2)
print(f"Ответ: {response2}")
print("-" * 50)

# # Вопрос 3: Вопрос, на который, возможно, нет прямого ответа в контексте
question3 = "Who is Moses?"
print(f"Вопрос: {question3}")
response3 = rag_chain.invoke(question3)
print(f"Ответ: {response3}")
print("-" * 50)

# # Вопрос 4: Вопрос о деталях сюжета
question4 = "Who was the serpent?"
print(f"Вопрос: {question4}")
response4 = rag_chain.invoke(question4)
print(f"Ответ: {response4}")
print("-" * 50)




--- Тестирование RAG-системы ---
Вопрос: Who was Uriah the Hittite?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Ответ: Используйте только следующий фрагмент контекста, чтобы ответить на вопрос пользователя.

Инструкции:

Извлеките все релевантные факты и информацию, относящиеся к вопросу.

На основе этих фактов, сформулируйте краткий, точный и информативный ответ, что известно о той или иной личности, предмете, обстоятельствах, месте, действиях, о которых спрашивается.

Если контекст не содержит достаточной информации для прямого ответа на вопрос, укажите это.

Контекст:
[Document(id='45b1cbb9-8008-4532-9a2b-8b630eb8663f', metadata={'source': 'your_source'}, page_content='assigned Uriah unto a place where he knew that valiant men were.\r\n\r\n11:17 And the men of the city went out, and fought with Joab: and\r\nthere fell some of the people of the servants of David; and Uriah the\r\nHittite died also.\r\n\r\n11:18 Then Joab sent and told David all the things concerning the war;\r\n11:19 And charged the messenger, saying, When thou hast made an end of'), Document(id='1e316bf8-9e49-4763-9142-196517

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Ответ: Используйте только следующий фрагмент контекста, чтобы ответить на вопрос пользователя.

Инструкции:

Извлеките все релевантные факты и информацию, относящиеся к вопросу.

На основе этих фактов, сформулируйте краткий, точный и информативный ответ, что известно о той или иной личности, предмете, обстоятельствах, месте, действиях, о которых спрашивается.

Если контекст не содержит достаточной информации для прямого ответа на вопрос, укажите это.

Контекст:
[Document(id='6a77a418-6b8c-419e-82f1-2e2db771a709', metadata={'source': 'your_source'}, page_content='from whence he was taken.\r\n\r\n3:24 So he drove out the man; and he placed at the east of the garden\r\nof Eden Cherubims, and a flaming sword which turned every way, to keep\r\nthe way of the tree of life.\r\n\r\n4:1 And Adam knew Eve his wife; and she conceived, and bare Cain, and\r\nsaid, I have gotten a man from the LORD.\r\n\r\n4:2 And she again bare his brother Abel. And Abel was a keeper of'), Document(id='6404647f-e3e

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Ответ: Используйте только следующий фрагмент контекста, чтобы ответить на вопрос пользователя.

Инструкции:

Извлеките все релевантные факты и информацию, относящиеся к вопросу.

На основе этих фактов, сформулируйте краткий, точный и информативный ответ, что известно о той или иной личности, предмете, обстоятельствах, месте, действиях, о которых спрашивается.

Если контекст не содержит достаточной информации для прямого ответа на вопрос, укажите это.

Контекст:
[Document(id='dce81d0d-665a-4164-882e-3f0d51e257ea', metadata={'source': 'your_source'}, page_content='Moses.\r\n\r\n35:30 And Moses said unto the children of Israel, See, the LORD hath\r\ncalled by name Bezaleel the son of Uri, the son of Hur, of the tribe\r\nof Judah; 35:31 And he hath filled him with the spirit of God, in\r\nwisdom, in understanding, and in knowledge, and in all manner of\r\nworkmanship; 35:32 And to devise curious works, to work in gold, and'), Document(id='fd8022b4-b8e5-414a-bc8a-c43858b4a967', metadata={'s

In [ ]:

# Пример 5: Вопрос о деталях сюжета
question5 = "Who was Nebuchadnezzar?"
print(f"Вопрос: {question5}")
response5 = rag_chain.invoke(question5)
print(f"Ответ: {response5}")
print("-" * 50)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Вопрос: Who was Nebuchadnezzar?
Ответ: Используйте только следующий фрагмент контекста, чтобы ответить на вопрос пользователя.

Инструкции:

Извлеките все релевантные факты и информацию, относящиеся к вопросу.

На основе этих фактов, сформулируйте краткий, точный и информативный ответ, что известно о той или иной личности, предмете, обстоятельствах, месте, действиях, о которых спрашивается.

Если контекст не содержит достаточной информации для прямого ответа на вопрос, укажите это.

Контекст:
[Document(id='28ecae26-9ada-4b4c-a32c-82dd09dc16cb', metadata={'source': 'your_source'}, page_content='the golden image that Nebuchadnezzar the king had set up.\r\n\r\n3:8 Wherefore at that time certain Chaldeans came near, and accused\r\nthe Jews.\r\n\r\n3:9 They spake and said to the king Nebuchadnezzar, O king, live for\r\never.\r\n\r\n3:10 Thou, O king, hast made a decree, that every man that shall hear\r\nthe sound of the cornet, flute, harp, sackbut, psaltery, and dulcimer,'), Document(id='2

In [ ]:

# Пример 5: Вопрос о деталях сюжета
question5 = "What was a name of Adam's wife"
print(f"Вопрос: {question5}")
response5 = rag_chain.invoke(question5)
print(f"Ответ: {response5}")
print("-" * 50)


In [ ]:

print("\nВывод по тестированию RAG-системы:")
print(" - Система успешно извлекает релевантный контекст и генерирует ответы.")
print(" - Для вопросов вне контекста базы знаний, LLM должна отвечать, что не может найти информацию.")

#
# ## 8. Оценка и оптимизация RAG-системы (концептуально)
#
# Оценка RAG-систем — это сложная задача, требующая как традиционных метрик NLP,
# так и метрик, специфичных для качества генерации и получения информации.
#
# **Метрики оценки (концептуально):**
# -   **Relevance (Релевантность):** Насколько ответ релевантен запросу.
# -   **Faithfulness (Достоверность):** Насколько ответ основан на предоставленном контексте.
# -   **Context Precision/Recall (Точность/Полнота контекста):** Насколько извлеченный контекст
#     был релевантен и полон.
# -   **Answer Correctness (Правильность ответа):** Насколько ответ верен фактически.
# -   **Fluency (Беглость), Coherence (Связность).**
#
# **Инструменты:** RAGAS (Rag-as-a-Service), TruLens, LangChain Evaluators (требуют размеченных данных).
#
# **Оптимизация:** Экспериментирование с:
# -   Различными стратегиями разбиения на чанки (размер, перекрытие).
# -   Моделями эмбеддингов.
# -   Порогами поиска в векторной базе данных (параметр `k` в ретривере).
# -   Шаблонами промптов для LLM.
# -   Моделями LLM (использование более крупных или специализированных моделей).
#
# Для полноценной оценки требуются размеченные пары "вопрос-ответ" с указанием релевантных фрагментов текста.
# В рамках этого ноутбука мы проводим только качественную оценку на примерах.

print("\n--- Концепция оценки и оптимизации конвейера RAG ---")
print("Для полноценной оценки RAG-системы требуются размеченные данные (вопросы, правильные ответы, релевантные чанки).")
print("Оптимизация включает эксперименты с параметрами разбиения на чанки, моделями эмбеддингов, LLM и промптами.")

#
# ## 9. Сохранение артефактов для развертывания
#
# Для развертывания RAG-системы в виде сервиса нам нужно сохранить обученные компоненты:
# -   Векторную базу данных FAISS.
# -   Название модели эмбеддингов (для загрузки в сервисе).
# -   Название LLM (для загрузки в сервисе).
# -   Шаблон промпта.

# Векторная база данных уже сохранена в разделе 4: /tmp/faiss_index_alice
# model_llm и tokenizer будут загружены по имени в сервисе.
# embeddings также будут загружены по имени.

# Сохраняем информацию о моделях для сервиса
rag_config = {
    "embedding_model_name": embedding_model_name,
    "llm_model_name": llm_model_name,
    "vector_store_path": vector_store_path,
    "prompt_template": template
}

import json
with open("/tmp/rag_config.json", "w") as f:
    json.dump(rag_config, f, indent=4)
print("Конфигурация RAG-системы сохранена в '/tmp/rag_config.json'.")

print("\nВывод по сохранению артефактов:")
print(" - Векторная база данных FAISS и конфигурация RAG-системы сохранены.")
print(" - Эти файлы будут использоваться для инициализации RAG-системы в FastAPI, Gradio и Streamlit сервисах.")